### FurryTagger
----

In [ ]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

%cd /
!git clone --depth 1 https://huggingface.co/Thouph/eva02-vit-large-448-8046 /eva02-vit-large-448-8046
%cd /eva02-vit-large-448-8046
!git clone https://huggingface.co/datasets/k4d3/ayaya ./images
!pip3 install -U timm

import json
import os
from PIL import Image
import torch
from torchvision.transforms import transforms

THRESHOLD = 0.3
root_dir = r'./images'  # Change the root directory to the desired starting point

model_path = r'./model.pth'

model = torch.load(model_path, map_location=torch.device('cuda'))
model.eval()

transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[
        0.48145466,
        0.4578275,
        0.40821073
    ], std=[
        0.26862954,
        0.26130258,
        0.27577711
    ])
])

with open("tags_8041.json", "r", encoding='utf8') as file:
    tags = json.load(file)
allowed_tags = sorted(tags)
allowed_tags.insert(0, "placeholder0")
allowed_tags.append("placeholder1")
allowed_tags.append("explicit")
allowed_tags.append("questionable")
allowed_tags.append("safe")

image_exts = ['.jpg', '.jpeg', '.png', '.gif']

# Recurse into subdirectories
image_files = [file for file in Path(root_dir).rglob('*') if file.suffix.lower() in image_exts]

for image_path in image_files:
    image_path_str = str(image_path)

    img = Image.open(image_path_str).convert('RGB')
    tensor = transform(img).unsqueeze(0).cuda()

    with torch.no_grad():
        out = model(tensor)

    probabilities = torch.sigmoid(out[0])
    indices = torch.where(probabilities > THRESHOLD)[0]
    values = probabilities[indices]

    sorted_indices = torch.argsort(values, descending=True)

    tags_to_write = [allowed_tags[indices[i]].replace("_", " ") for i in sorted_indices if allowed_tags[indices[i]] not in ("placeholder0", "placeholder1")]

    text_filename = image_path.stem + ".txt"
    text_path = image_path.parent / text_filename

    with open(text_path, "w", encoding='utf8') as text_file:
        text_file.write(", ".join(tags_to_write))

In [ ]:
!zip -r ./output.zip ./images/**/*.txt ./images/*.txt
!mv ./output.zip /content/drive/MyDrive/output.zip